<h3>se comienza a desarrollar el proyecto y se que se necesita crear la base de datos de la situiacion en Chicago</h3>

In [2]:
import MySQLdb

db=MySQLdb.connect(host="localhost"
                ,port=3306,
                user="chaki"
                ,password="timecha12"
                ,database="Mysql_Learners")




<h2>problem 1</h2>
<p>find the total number of crimes recorded in the crime table</p>

In [48]:
cursor = db.cursor()
cursor.execute("select count(*) from chicago_crime;") # ejecuta sentencia SQL
data = cursor.fetchone() # recupera primer registro
print(data[0])

533


<h2>problem 2</h2>
<p>List community areas with per capita income less than 11000.</p>

In [69]:
lista=[]
cursor = db.cursor()
cursor.execute("select COMMUNITY_AREA_NAME from chicago_socioeconomic_data where PER_CAPITA_INCOME<11000;") # ejecuta sentencia SQL
data = cursor.fetchall() # recupera primer registro
for i in range(len(data)):
    lista.append(data[i][0])
lista


['West Garfield Park', 'South Lawndale', 'Fuller Park', 'Riverdale']

<h2>problem 3</h2>
<p>List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)</p>

In [22]:
cursor = db.cursor()
df=pd.read_sql("select CASE_NUMBER from chicago_crime where description like '%MINOR%';",db)
df


/home/chaki/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CASE_NUMBER
0,HL266884
1,HK238408


<h2>problem 4</h2>
<p>List all kidnapping crimes involving a child?</p>

In [24]:
import pandas as pd

cursor = db.cursor()
df=pd.read_sql("select * from chicago_crime where PRIMARY_TYPE = 'KIDNAPPING' AND description like '%CHILD%';",db)
df


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,...,15,29,25,20,1143050,1897546,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


<h2>problem 5</h2>
<p>What kinds of crimes were recorded at schools?</p>

In [25]:
cursor = db.cursor()
df=pd.read_sql("select PRIMARY_TYPE, LOCATION_DESCRIPTION from chicago_crime where LOCATION_DESCRIPTION LIKE 'SCHOOL%' ;",db)
df



/home/chaki/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,PRIMARY_TYPE,LOCATION_DESCRIPTION
0,BATTERY,"SCHOOL, PUBLIC, GROUNDS"
1,BATTERY,"SCHOOL, PUBLIC, BUILDING"
2,CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
3,NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
4,NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
5,ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
6,CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
7,PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
8,PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


<h2>problem 6</h2>
<p>List the average safety score for all types of schools.</p>

In [28]:
cursor = db.cursor()
df=pd.read_sql("select `Elementary, Middle, or High School` as TYPE_SCHOOL, AVG(SAFETY_SCORE) as AVERAGE_SAFTY_SCORE\
        from chicago_public_schools \
            GROUP BY(`Elementary, Middle, or High School`);",db)
df


,TYPE_SCHOOL,AVERAGE_SAFTY_SCORE
0,ES,44.696970
1,HS,45.354839
2,MS,48.000000


<h2>problem 7</h2>
<p>List 5 community areas with highest % of households below poverty line.</p>

In [12]:
cursor = db.cursor()
df=pd.read_sql("SELECT COMMUNITY_AREA_NAME \
    FROM `chicago_socioeconomic_data` \
        ORDER BY `chicago_socioeconomic_data`.`PERCENT_HOUSEHOLDS_BELOW_POVERTY` DESC LIMIT 5",db)
df


/home/chaki/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,COMMUNITY_AREA_NAME
0,Riverdale
1,Fuller Park
2,Englewood
3,North Lawndale
4,East Garfield Park


<h2>problem 8</h2>
<p>Which community area is most crime prone?</p>

In [29]:
cursor = db.cursor()
df=pd.read_sql("SELECT COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) as TOTAL \
    FROM chicago_crime GROUP BY COMMUNITY_AREA_NUMBER ORDER BY TOTAL DESC LIMIT 1",db)
df


/home/chaki/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,COMMUNITY_AREA_NUMBER,TOTAL
0,25,43


<h2>problem 9</h2>
<p>Use a sub-query to find the name of the community area with highest hardship index</p>

In [33]:
cursor = db.cursor()
df=pd.read_sql("SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM\
     chicago_socioeconomic_data\
         WHERE HARDSHIP_INDEX =(SELECT MAX(HARDSHIP_INDEX) from chicago_socioeconomic_data);",db)
df


,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
0,Riverdale,98


<h2>problem 10</h2>
<p>Use a sub-query to determine the Community Area Name with most number of crimes?</p>

In [20]:
cursor = db.cursor()
df=pd.read_sql("SELECT e.COMMUNITY_AREA_NAME, c.TOTAL as TOTAL_CRIME_AREA \
    FROM chicago_socioeconomic_data e INNER JOIN \
        (SELECT COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) as TOTAL \
            FROM chicago_crime GROUP BY COMMUNITY_AREA_NUMBER ORDER BY TOTAL DESC LIMIT 1) c \
                on e.COMMUNITY_AREA_NUMBER = c.COMMUNITY_AREA_NUMBER;",db)
df


/home/chaki/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,COMMUNITY_AREA_NAME,TOTAL_CRIME_AREA
0,Austin,43
